## Step1：讀檔案轉向量並匯入DB

### 使用文件載入器

In [1]:
from langchain.document_loaders import PyMuPDFLoader

# 初始化 PyMuPDFLoader 物件，並加載指定的 PDF 文件
loader = PyMuPDFLoader("Virtual_characters.pdf")

# 使用 loader 的 load() 方法從 PDF 中提取文本數據，
# 會返回包含 PDF 內容，將每一頁的文本提取出來。
PDF_data = loader.load()

### 使用 Text splitter 分割文件

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 初始化文本分割器，設置參數：
# - chunk_size=100：每個文本塊最大的長度（以字符數計算）。
# - chunk_overlap=5：每兩個連續文本塊之間的重疊字符數。
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=5)

# 使用文本分割器對文檔（PDF_data）進行分割。
# PDF_data 從 PDF 文件中提取的原始文本。
all_splits = text_splitter.split_documents(PDF_data)

### 載入 Embedding model
將上步驟分割的 chunk 文字轉換為向量


In [3]:
from langchain.embeddings import HuggingFaceEmbeddings

# 設定要使用的預訓練模型名稱
# "sentence-transformers/all-MiniLM-L6-v2" 是一個預訓練的句子嵌入模型，這個模型可用來將文本轉換為向量。
# sentence-transformers 庫中的 "all-MiniLM-L6-v2" 模型，這是一個適用於句子嵌入的輕量級模型，能夠將句子轉換為固定長度的向量
model_name = "sentence-transformers/all-MiniLM-L6-v2"

# model_kwargs 是一個字典，其中我們指定在 CPU 上運行，而不是 GPU，這對於沒有 GPU 的環境來說很重要。
model_kwargs = {'device': 'cpu'}

# 初始化 HuggingFaceEmbeddings 物件，並傳入模型名稱和參數
# 這將加載並初始化指定的嵌入模型，用來將文本轉換為向量。
# HuggingFaceEmbeddings 是 LangChain 庫中的一個類，它能夠使用 HuggingFace Hub 上的模型來生成文本嵌入（即將文本轉換為向量），這些向量可以用來做檢索、相似度計算或其他自然語言處理任務
embedding = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

C:\Users\User\AppData\Local\Temp\ipykernel_166140\992490538.py:14: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)
c:\Users\User\anaconda3\envs\rag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 將 Embedding 結果匯入 VectorDB
Embedding 後的結果我們會儲存在 VectorDB 中，常見的 VectorDB 有 Chroma、Pinecone、FAISS等

In [4]:
from langchain.vectorstores import Chroma

# 設定儲存向量的資料庫目錄
persist_directory = 'db'  # 這個資料夾用來存儲 Chroma 向量資料庫

# 使用 Chroma 將嵌入的文本存儲到向量資料庫中
# - documents=all_splits：之前已經分割的文檔列表。
# - embedding=embedding：HuggingFaceEmbeddings 將文本轉換的向量。
# - persist_directory=persist_directory：儲存資料庫的目錄位置，這裡儲存向量資料庫。
vectordb = Chroma.from_documents(documents=all_splits, embedding=embedding, persist_directory=persist_directory)


## Step2：啟用 LLM 服務

方法 1: 使用 LangChain 的 LlamaCpp 接口啟動 Llama2 服務，由 LangChain 幫你把 llama2 服務啟動，不需要手動啟動服務

方法 2: 使用 llama.cpp 啟動 Llama2 服務並提供 API，可更靈活控制服務將其與其他應用集成

### 使用 LangChain 的 LlamaCpp

In [3]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import LlamaCpp

# 設定模型路徑
model_path = r"C:\Users\User\Downloads\中山\資管所\論文\程式實驗\RAG實驗\llama.cpp\models\ggml-vocab-aquila.gguf"

# 初始化 LlamaCpp 物件來加載並配置模型
llm = LlamaCpp(
    model_path=model_path,  # 指定 Llama3.3 模型的路徑
    n_gpu_layers=100,       # 指定 GPU 層的數量
    n_batch=512,            # 設定批次大小
    n_ctx=2048,             # 設定上下文窗口大小
    f16_kv=True,            # 是否使用 16 位浮點數來存儲鍵值對
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),  # 設置回調處理器
    verbose=True,           # 設置為 True 以顯示詳細的調試信息
)

# 使用模型進行推理
prompt = "請解釋量子計算的基本概念"
response = llm(prompt)

print(response)


llama_model_loader: loaded meta data with 18 key-value pairs and 0 tensors from C:\Users\User\Downloads\中山\資管所\論文\程式實驗\RAG實驗\llama.cpp\models\ggml-vocab-aquila.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = D:\Diverses\models
llama_model_loader: - kv   2:                       llama.context_length u32              = 2048
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   

ValidationError: 1 validation error for LlamaCpp
  Value error, Could not load Llama model from path: C:\Users\User\Downloads\中山\資管所\論文\程式實驗\RAG實驗\llama.cpp\models\ggml-vocab-aquila.gguf. Received error Failed to load model from file: C:\Users\User\Downloads\中山\資管所\論文\程式實驗\RAG實驗\llama.cpp\models\ggml-vocab-aquila.gguf [type=value_error, input_value={'model_path': 'C:\\Users...: None, 'grammar': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

In [4]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(openai_api_key='None', openai_api_base='http://127.0.0.1:8080/v1')

C:\Users\User\AppData\Local\Temp\ipykernel_3756\659352066.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(openai_api_key='None', openai_api_base='http://127.0.0.1:8080/v1')


In [5]:
from langchain.chains import LLMChain
from langchain.chains.prompt_selector import ConditionalPromptSelector
from langchain.prompts import PromptTemplate

DEFAULT_LLAMA_SEARCH_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""<<SYS>> \n You are an assistant tasked with improving Google search \
results. \n <</SYS>> \n\n [INST] Generate THREE Google search queries that \
are similar to this question. The output should be a numbered list of questions \
and each should have a question mark at the end: \n\n {question} [/INST]""",
)

DEFAULT_SEARCH_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an assistant tasked with improving Google search \
results. Generate THREE Google search queries that are similar to \
this question. The output should be a numbered list of questions and each \
should have a question mark at the end: {question}""",
)

QUESTION_PROMPT_SELECTOR = ConditionalPromptSelector(
    default_prompt=DEFAULT_SEARCH_PROMPT,
    conditionals=[(lambda llm: isinstance(llm, LlamaCpp), DEFAULT_LLAMA_SEARCH_PROMPT)],
)

prompt = QUESTION_PROMPT_SELECTOR.get_prompt(llm)
prompt

PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='You are an assistant tasked with improving Google search results. Generate THREE Google search queries that are similar to this question. The output should be a numbered list of questions and each should have a question mark at the end: {question}')

In [6]:
llm_chain = LLMChain(prompt=prompt, llm=llm)
question = "What is Taiwan known for?"
llm_chain.invoke({"question": question})

C:\Users\User\AppData\Local\Temp\ipykernel_3756\424679987.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(prompt=prompt, llm=llm)


APIConnectionError: Connection error.